In [2]:
import numpy as np
import pandas as pd


In [3]:
df = pd.read_csv("train.csv")
df

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


0         1
1         1
2         1
3         1
4         1
         ..
912995    2
912996    2
912997    2
912998    2
912999    2
Length: 913000, dtype: int64

In [22]:
#more preprocessing 
df['store'] & df['item'] 
#Convert 0 to Nan
df = df.loc[(df!=0).any(axis=1)]
df = df.dropna()

train_target = df['sales'].to_numpy()
train_features = df.drop(['sales','date'], axis = 1).to_numpy()

print("Target Shape : ", train_target.shape)
print("Featuers Shape : ", train_features.shape)
train_features

Target Shape :  (913000,)
Featuers Shape :  (913000, 2)


array([[ 1,  1],
       [ 1,  1],
       [ 1,  1],
       ...,
       [10, 50],
       [10, 50],
       [10, 50]])

In [23]:
import math
# Sigmoid function
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [24]:
# Loss function for logistic regression
def loss(y, y_pred):
    loss = -np.mean(y*(np.log(y_pred)) - (1-y)*np.log(1-y_pred))
    return loss

In [25]:
def gradients(X, y, y_pred): 
    # X is the input, y is the target value, y_pred is the prediction, w is the weight and b is the bias.
    
    m = X.shape[0]
    dw = (1/m)*np.dot(X.T, (y_pred - y))
    db = (1/m)*np.sum((y_pred - y)) 
    
    return dw, db

In [26]:
def train(X, y, bs, epochs, lr):
    # X is the input, y is the target value, y_pred is the prediction, 
    # bs is the batch sizem epochs is the number of iterations and lr is the learning rate.

    m, n = X.shape
    
    # Initialize weights and bias to zeros.
    w = np.zeros((n,1))
    b = 0
    
    # Reshaping y.
    y = y.reshape(m,1)
    
    # Normalize the inputs.
    #x = normalize(X)
    
    # Initialize an empty list to store the losses.
    losses = []
    
    # Training
    for epoch in range(epochs):
        for i in range((m-1)//bs + 1):
            
            # Define the batches.
            start_i = i*bs
            end_i = start_i + bs
            xb = X[start_i:end_i]
            yb = y[start_i:end_i]
            
            # Calculate the prediction.
            y_pred = sigmoid(np.dot(xb, w) + b)
            
            # Getting the gradients of loss with respect to the parameters.
            dw, db = gradients(xb, yb, y_pred)
            
            # Update the parameters.
            w -= lr*dw
            b -= lr*db
        
        # Calculating loss and appending it in the list.
        l = loss(y, sigmoid(np.dot(X, w) + b))
        losses.append(l)
        
    return w, b, losses

In [27]:
def predict(X,w,b):
    
    # X --> Input.
    
    # Normalizing the inputs.
    # x = normalize(X)
    
    # Calculating predictions.
    preds = sigmoid(np.dot(X, w) + b)
    
    # Empty List to store predictions.
    pred_class = []
    # if y_pred >= 0.5 --> round up to 1
    # if y_pred < 0.5 --> round up to 1
    pred_class = [1 if i > 0.5 else 0 for i in preds]
    
    return np.array(pred_class)

In [28]:
w, b, l = train(train_features, train_target, bs=100, epochs=1000, lr=0.01)

/var/folders/6q/kc0wrvjn5kd1pn_f5j3vl78w0000gn/T/ipykernel_29655/2851178450.py:3: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(y*(np.log(y_pred)) - (1-y)*np.log(1-y_pred))
/var/folders/6q/kc0wrvjn5kd1pn_f5j3vl78w0000gn/T/ipykernel_29655/2851178450.py:3: RuntimeWarning: invalid value encountered in multiply
  loss = -np.mean(y*(np.log(y_pred)) - (1-y)*np.log(1-y_pred))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:179: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


In [11]:
def accuracy(y, y_pred):
    accuracy = np.sum(y == y_pred) / len(y)
    return accuracy

In [37]:
print(accuracy(train_target, predict(train_features, w, b)))


0.406762104283054


In [40]:
from sklearn.metrics import f1_score 
pred = predict(train_features, w, b) 
f1_score(train_target, pred, average='macro')




0.1918612325786324


In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(train_target, pred)